<h1> RQ4 </h1>
This code is written by Santo Palaia (1st part) and Pooja Jambaladinni (2nd part)

<h6> PART 1 <h6>

In [1]:
#import libraries
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load the json files
event = pd.read_json(r'/users/poojacj/Desktop/adm_hw2/events/events_England.json')[['eventId', 'tags', 'playerId','eventSec','matchId']]
#consider only pass
pass_event_all=event.loc[event['eventId']==8] 

In [3]:
pass_event_all

,eventId,tags,playerId,eventSec,matchId
0,8,[{'id': 1801}],25413,2.758649,2499719
1,8,[{'id': 1801}],370224,4.946850,2499719
2,8,[{'id': 1801}],3319,6.542188,2499719
3,8,[{'id': 1801}],120339,8.143395,2499719
4,8,[{'id': 1801}],167145,10.302366,2499719
...,...,...,...,...,...
643134,8,[{'id': 1801}],8313,2760.286946,2500098
643135,8,[{'id': 1801}],379209,2763.698599,2500098
643141,8,[{'id': 1801}],8561,2783.137836,2500098
643142,8,[{'id': 1801}],20620,2785.702032,2500098


In [4]:
#drop column eventId
pass_event_all.drop('eventId', axis=1, inplace=True) #we don't need eventId

In [5]:
myList=[]
for index, row in pass_event_all.iterrows():
    myList.append(1) if {'id':1801} in row['tags'] else myList.append(0) #create a list of 1 (pass compleate) and 0 (pass not totaly complete)

In [6]:
pass_event_all['pass']=myList #add this column
pass_event_all.drop('tags', axis=1, inplace=True) #we don't need tags

In [7]:
pass_event_all

,playerId,eventSec,matchId,pass
0,25413,2.758649,2499719,1
1,370224,4.946850,2499719,1
2,3319,6.542188,2499719,1
3,120339,8.143395,2499719,1
4,167145,10.302366,2499719,1
...,...,...,...,...
643134,8313,2760.286946,2500098,1
643135,379209,2763.698599,2500098,1
643141,8561,2783.137836,2500098,1
643142,20620,2785.702032,2500098,1


In [8]:
pass_event_final=pass_event_all.groupby('playerId').agg({'pass':['sum', 'count']}) #grouping on playerId and aggregating on sum(number of comlete pass) and count of total pass tried
pass_event_final.columns=['_'.join(col).strip() for col in pass_event_final.columns.values] #redefination of columns
pass_event_final=pass_event_final.reset_index() #reset the index

In [9]:
pass_event_final

,playerId,pass_sum,pass_count
0,0,0,6
1,36,790,898
2,38,192,224
3,48,2113,2370
4,54,1803,2196
...,...,...,...
509,448708,5,6
510,450826,12,17
511,486252,165,226
512,531655,7,11


<h6> PART 2 <h6>

In [10]:
pass_event_final["ratio"] = pass_event_final["pass_sum"] / pass_event_final["pass_count"]

In [11]:
pass_event_final

,playerId,pass_sum,pass_count,ratio
0,0,0,6,0.000000
1,36,790,898,0.879733
2,38,192,224,0.857143
3,48,2113,2370,0.891561
4,54,1803,2196,0.821038
...,...,...,...,...
509,448708,5,6,0.833333
510,450826,12,17,0.705882
511,486252,165,226,0.730088
512,531655,7,11,0.636364


In [12]:
#sort the columns in descending order of ratio
pass_event_final_sorted = pass_event_final.sort_values(by='ratio', ascending=False)

In [13]:
pass_event_final_sorted.head(15)

,playerId,pass_sum,pass_count,ratio
513,532949,1,1,1.000000
506,447538,1,1,1.000000
504,447254,4,4,1.000000
500,423162,18,18,1.000000
497,415747,6,6,1.000000
480,353241,4,4,1.000000
471,340476,1,1,1.000000
459,291528,7,7,1.000000
418,227756,4,4,1.000000
400,134708,1,1,1.000000


In [14]:
#select the top rows with maximun value of ratio
df_top_players = pass_event_final_sorted[pass_event_final_sorted["ratio"] == 1]

In [15]:
df_top_players

,playerId,pass_sum,pass_count,ratio
513,532949,1,1,1.0
506,447538,1,1,1.0
504,447254,4,4,1.0
500,423162,18,18,1.0
497,415747,6,6,1.0
480,353241,4,4,1.0
471,340476,1,1,1.0
459,291528,7,7,1.0
418,227756,4,4,1.0
400,134708,1,1,1.0


In [17]:
df_result = df_top_players[['playerId','ratio']]

In [18]:
df_result.columns = ['PlayerId','RatioPasses']

In [20]:
#save the results to a csv file
df_result.to_csv("/users/poojacj/Desktop/adm_hw2/RQ4/highest_ratio_of_passes.csv",index=None)